In [3]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
import seaborn as sns
from shapely.geometry import Point, LineString, MultiPoint
import numpy as np
import contextily as ctx
import requests
from io import StringIO, BytesIO
import json
import datetime as dt
from ast import literal_eval
from shapely.wkt import loads
import plotly.express as px
from dotenv import load_dotenv, find_dotenv

In [2]:
# API KEY
load_dotenv(find_dotenv())
FOOTBALL = os.environ.get("FOOTBALL")

In [8]:
# headers = {'X-Auth-Token': FOOTBALL}
# url = 'https://api.football-data.org/v2/matches'
# params = {'dateFrom': '2018-04-14',
#          'dateTo': '2018-04-16'}
# r = requests.get(url, headers=headers, params=params)
# r.json()

{'message': 'The resource you are looking for is restricted. Please pass a valid API token and check your subscription for permission.',
 'errorCode': 403}

In [14]:
root = 'raw_data/football/'
dfs = []
for i in os.listdir(root):
    if '.csv' in i:
        df = pd.read_csv(root+i)
        dfs.append(df)

In [16]:
# filter all by juve
juve1 = dfs[0][(dfs[0]['HomeTeam'] == 'Juventus') \
              | (dfs[0]['AwayTeam'] == 'Juventus')]['Date']
juve1 = pd.to_datetime(juve1, format='%d/%m/%Y')

juve2 = dfs[1][(dfs[1]['Home Team'] == 'Juventus') \
               | (dfs[1]['Away Team'] == 'Juventus')]['Date']
juve2 = pd.to_datetime(juve2.apply(lambda x: x[:10]), format="%d/%m/%Y")

juve3 = dfs[2][(dfs[2]['HomeTeam'] == 'Juventus') \
              | (dfs[2]['AwayTeam'] == 'Juventus')]['Date']
juve3 = pd.to_datetime(juve3, format='%d/%m/%y')

juve4 = dfs[3][(dfs[3]['Home Team'] == 'Juventus') \
               | (dfs[3]['Away Team'] == 'Juventus')]['Date']
juve4 = pd.to_datetime(juve4.apply(lambda x: x[:10]), format="%d/%m/%Y")

In [18]:
# concat all dates
all_concat = pd.DataFrame(pd.concat([juve1, juve2, juve3, juve4]))
# all_concat['Date'] = pd.to_datetime(all_concat.Date)
all_concat['is_match'] = 1

In [19]:
all_concat.sort_values(by='Date')

,Date,is_match
0,2017-08-19,1
11,2017-08-26,1
20,2017-09-09,1
0,2017-09-12,1
36,2017-09-17,1
...,...,...
331,2019-04-27,1
340,2019-05-03,1
354,2019-05-12,1
365,2019-05-19,1


In [20]:
# get all 2018 matches
all_concat_2018 = all_concat[(all_concat.Date > '01-01-2018') \
                            & (all_concat.Date < '2018-12-31')]


In [25]:
# put in 2018 time series
r = pd.date_range('2018-01-01', '2018-12-31', freq='h')
matches = all_concat_2018.set_index('Date').reindex(r).rename_axis('Date').reset_index()

In [29]:
matches.to_csv('raw_data/football/matches_2018.csv')

In [27]:
all_concat_2018.sort_values(by='Date')

,Date,is_match
191,2018-01-06,1
207,2018-01-22,1
210,2018-01-27,1
226,2018-02-04,1
230,2018-02-09,1
6,2018-02-13,1
248,2018-02-18,1
259,2018-03-03,1
7,2018-03-07,1
270,2018-03-11,1


In [28]:
matches

,Date,is_match
0,2018-01-01 00:00:00,NaN
1,2018-01-01 01:00:00,NaN
2,2018-01-01 02:00:00,NaN
3,2018-01-01 03:00:00,NaN
4,2018-01-01 04:00:00,NaN
...,...,...
8732,2018-12-30 20:00:00,NaN
8733,2018-12-30 21:00:00,NaN
8734,2018-12-30 22:00:00,NaN
8735,2018-12-30 23:00:00,NaN
